# Customer Lifetime Value (CLV) Analysis

This notebook explores Customer Lifetime Value (CLV) within the synthetic pet insurance dataset.

CLV helps answer key business questions:
- Which customers generate the most value over time?
- How do species, breeds, and products influence long-term profitability?
- Are certain customer segments more likely to renew, claim frequently, or churn?
- How does claim behaviour impact lifetime value?

Data we are using:
1. Total premiums paid
2. Total claims cost
3. Policy duration
4. Renewal behaviour
5. Net lifetime value (premiums – claims)

This notebook forms part of the analytical layer of the project.


In [2]:
import pandas as pd
import numpy as np

# Load datasets
customers = pd.read_csv(r"C:\Users\leebe\Documents\VScode\portfolio project\pet_data_project\data\raw\customers.csv")
pets = pd.read_csv(r"C:\Users\leebe\Documents\VScode\portfolio project\pet_data_project\data\raw\pets.csv")
policies = pd.read_csv(r"C:\Users\leebe\Documents\VScode\portfolio project\pet_data_project\data\raw\policies.csv")
products = pd.read_csv(r"C:\Users\leebe\Documents\VScode\portfolio project\pet_data_project\data\raw\products.csv")
claims = pd.read_csv(r"C:\Users\leebe\Documents\VScode\portfolio project\pet_data_project\data\raw\claims.csv")
claim_payments = pd.read_csv(r"C:\Users\leebe\Documents\VScode\portfolio project\pet_data_project\data\raw\claim_payments.csv")

print("Datasets loaded successfully.")

Datasets loaded successfully.


# Build CLV Base Table

In [3]:
# Merge policies with products to get premium + limits
policy_product = policies.merge(products, on="product_id", how="left")

# Merge pets to link customers
policy_pet = policy_product.merge(pets[["pet_id", "customer_id", "species", "breed"]], on="pet_id", how="left")

# Merge customers
policy_customer = policy_pet.merge(customers, on="customer_id", how="left")

# Add policy duration in years
policy_customer["policy_duration_years"] = (
    pd.to_datetime(policy_customer["end_date"]) - pd.to_datetime(policy_customer["start_date"])
).dt.days / 365

# Premium paid over lifetime
policy_customer["total_premium_paid"] = policy_customer["monthly_premium"] * 12 * policy_customer["policy_duration_years"]

# Claims paid (sum of payments)
claim_costs = claim_payments.groupby("claim_id")["amount_paid"].sum().reset_index()
claims_with_costs = claims.merge(claim_costs, on="claim_id", how="left").fillna(0)

# Total claims per policy
policy_claims = claims_with_costs.groupby("policy_id")["amount_paid"].sum().reset_index()
policy_claims.rename(columns={"amount_paid": "total_claims_paid"}, inplace=True)

# Merge into main table
clv = policy_customer.merge(policy_claims, on="policy_id", how="left").fillna(0)

# Net Lifetime Value
clv["net_lifetime_value"] = clv["total_premium_paid"] - clv["total_claims_paid"]

clv.head()


,policy_id,pet_id,product_id,start_date,end_date,age_at_policy_start,active,product_name,coverage_type,annual_limit,...,email,phone,address,city,country,created_at,policy_duration_years,total_premium_paid,total_claims_paid,net_lifetime_value
0,1,30289,3,2022-03-19,2024-03-08,1,0,Lifetime Premier,Lifetime,1000,...,lromero@example.net,+1-640-864-1667x881,"01585 Hill Flat, East Barbara, AZ 52829",Richardside,Russian Federation,2018-08-20,1.972603,1359.675616,777.81,581.865616
1,2,45173,6,2021-11-17,2024-11-03,13,0,Lifetime Supreme,Lifetime,8000,...,gomezamy@example.org,+1-444-336-0391,"23595 Sarah Landing Apt. 630, Patrickburgh, AK...",East Sean,Mayotte,2023-01-25,2.964384,1073.936877,541.63,532.306877
2,3,18891,15,2025-07-19,2026-04-08,1,1,Max Benefit 10k,Maximum Benefit,12000,...,charleswright@example.net,001-209-630-5685x3931,"9688 Moore Islands, Buchananfurt, AS 18532",Roytown,Cayman Islands,2018-03-24,0.720548,277.814466,1455.23,-1177.415534
3,4,53368,4,2025-02-02,2025-09-01,3,0,Lifetime Elite,Lifetime,2000,...,alan59@example.net,001-698-871-0539x9406,"4702 Irwin Flats Apt. 202, Clayberg, NC 64051",Woodardberg,Luxembourg,2020-12-20,0.578082,622.108932,2387.88,-1765.771068
4,5,70238,10,2023-07-11,2024-10-05,7,0,Time-Limited Premium,Time-Limited,8000,...,ballardanita@example.org,3685125929,"28570 Tiffany Drives Apt. 942, Castroburgh, LA...",East Jane,Denmark,2020-05-29,1.238356,944.073205,1015.65,-71.576795


# Aggregate CLV at Customer Level

In [4]:
customer_clv = clv.groupby("customer_id").agg({
    "total_premium_paid": "sum",
    "total_claims_paid": "sum",
    "net_lifetime_value": "sum",
    "policy_id": "count",
    "pet_id": "nunique"
}).rename(columns={
    "policy_id": "num_policies",
    "pet_id": "num_pets"
}).reset_index()

customer_clv["avg_policy_value"] = customer_clv["net_lifetime_value"] / customer_clv["num_policies"]

customer_clv.head()


,customer_id,total_premium_paid,total_claims_paid,net_lifetime_value,num_policies,num_pets,avg_policy_value
0,2,1191.481644,0.00,1191.481644,1,1,1191.481644
1,4,408.044712,109.16,298.884712,1,1,298.884712
2,5,3524.659397,402.63,3122.029397,2,1,1561.014699
3,7,1840.854247,2330.46,-489.605753,3,1,-163.201918
4,8,4658.462466,1232.67,3425.792466,3,2,1141.930822


In [5]:
summary = f"""
Customer Lifetime Value Summary
--------------------------------
Total customers analysed: {customer_clv.shape[0]:,}

Average lifetime premium paid: £{customer_clv['total_premium_paid'].mean():.2f}
Average lifetime claims paid: £{customer_clv['total_claims_paid'].mean():.2f}
Average net lifetime value: £{customer_clv['net_lifetime_value'].mean():.2f}

Highest CLV customer: £{customer_clv['net_lifetime_value'].max():.2f}
Lowest CLV customer: £{customer_clv['net_lifetime_value'].min():.2f}

Average number of policies per customer: {customer_clv['num_policies'].mean():.2f}
Average number of pets per customer: {customer_clv['num_pets'].mean():.2f}
"""

print(summary)


Customer Lifetime Value Summary
--------------------------------
Total customers analysed: 43,729

Average lifetime premium paid: £2616.37
Average lifetime claims paid: £2635.49
Average net lifetime value: £-19.12

Highest CLV customer: £10708.45
Lowest CLV customer: £-21467.34

Average number of policies per customer: 2.74
Average number of pets per customer: 1.37

